In [1]:
# 🔧 EDIT THIS ONCE
BASE_DIR = "C:/Users/jacob/OneDrive/Documents/GitHub/external_data/nlp4neuro/results_april30/experiment_6"


In [2]:
# Task 1 – save plane-0 calcium image  ➜ fish11_calcium.pdf
import os, glob, tifffile, numpy as np, matplotlib.pyplot as plt

fish        = 11                                # ← change if needed
img_dir     = os.path.join(os.path.dirname(BASE_DIR), os.pardir, f"fish{fish}_images")
print(img_dir)
plane0_file = glob.glob(os.path.join(img_dir, "plane_0.*"))[0]   # .tif / .tiff / .png …

im  = tifffile.imread(plane0_file).astype(float)
im  = (im - im.min()) / (im.max() - im.min())                   # 0-1 normalise
im *= 1.5                                                       # brighten a little
im  = np.clip(im, 0, 1)

fig, ax = plt.subplots(figsize=(6,6), dpi=300)
ax.imshow(im, cmap="viridis")
ax.text(0.01, 0.99, "GCaMP6s", transform=ax.transAxes,
        va="top", ha="left", color="white", weight="bold", fontsize=16)
ax.axis("off")
out = os.path.join(BASE_DIR, f"fish{fish}_calcium.pdf")
fig.savefig(out, bbox_inches="tight", transparent=True)
plt.close(fig)
print("saved →", out)


C:/Users/jacob/OneDrive/Documents/GitHub/external_data/nlp4neuro/results_april30\..\fish11_images
saved → C:/Users/jacob/OneDrive/Documents/GitHub/external_data/nlp4neuro/results_april30/experiment_6\fish11_calcium.pdf


In [20]:
# Task 2 – red circles on most-salient neurons  ➜ fish11_overlay.pdf
import os, glob, ast, numpy as np, pandas as pd, matplotlib.pyplot as plt, tifffile

fish     = 11
TOP_K    = 30                                   # how many neurons to draw
DOT_MIN, DOT_MAX = 50, 300

# helpers ---------------------------------------------------------------------
def bright_plane0(f, boost=1.5):
    img_dir  = os.path.join(os.path.dirname(BASE_DIR), os.pardir, f"fish{f}_images")
    plane0   = glob.glob(os.path.join(img_dir, "plane_0.*"))[0]
    im       = tifffile.imread(plane0).astype(float)
    im       = (im-im.min())/(im.max()-im.min())
    return np.clip(im*boost,0,1)

def load_saliency(f):
    # ❶ correct root – one experiment_6, not two
    root = os.path.join(BASE_DIR, f"fish{f}")      # ← FIXED
    print(root)
    vecs = [
        np.load(os.path.join(r, "importance.npy"))
        for r, _, fs in os.walk(root) if "importance.npy" in fs
    ]
    if not vecs:                                   # nice-to-have guard
        raise FileNotFoundError(
            f"No importance.npy files under {root}. "
            "Check the folder structure or BASE_DIR."
        )
    return np.vstack(vecs).mean(0)


def plane0_coords(f):
    """Return (N×2) array of x-y coordinates + matching indices for plane_0."""
    h5 = os.path.join(os.path.dirname(BASE_DIR), os.pardir,
                      f"fish{f}_images", "functional_types_df.h5")
    h5 = os.path.normpath(h5)                  # tidy “../” segments
    df  = pd.read_hdf(h5)
    df0 = df[df.plane == "plane_0"]

    # ----- FIX: only literal_eval when value is a string ---------------------
    coords = np.vstack(
        df0.neur_coords.apply(
            lambda v: ast.literal_eval(v) if isinstance(v, str) else v
        )
    )
    # ------------------------------------------------------------------------
    return coords, df0.index.values.astype(int)

# data ------------------------------------------------------------------------
sal = load_saliency(fish)
coords, idx = plane0_coords(fish)               # coords all plane-0 neurons
sal0 = sal[idx]

top   = np.argsort(sal0)[-TOP_K:][::-1]
# after computing `sal0` and `top`:
arr = sal0[top]
sizes = (arr - arr.min()) / (np.ptp(arr) + 1e-9)
sizes = sizes * (DOT_MAX - DOT_MIN) + DOT_MIN

bg = bright_plane0(fish)
fig, ax = plt.subplots(figsize=(6,6))
ax.text(0.01, 0.99, f"Fish {fish}", transform=ax.transAxes,
        va="top", ha="left", color="white", weight="bold", fontsize=16)
ax.imshow(bg, cmap="gray", vmin=0, vmax=1)
ax.scatter(coords[top,0], coords[top,1], s=sizes,
           c="red", edgecolors="white", alpha=.8)
ax.axis("off"); fig.patch.set_alpha(0); ax.patch.set_alpha(0)

out = os.path.join(BASE_DIR, f"fish{fish}_overlay.pdf")
fig.savefig(out, bbox_inches="tight", transparent=True)
plt.close(fig)
print("saved →", out)


C:/Users/jacob/OneDrive/Documents/GitHub/external_data/nlp4neuro/results_april30/experiment_6\fish11
saved → C:/Users/jacob/OneDrive/Documents/GitHub/external_data/nlp4neuro/results_april30/experiment_6\fish11_overlay.pdf


In [28]:
# Task 3 – colour Pt, Hb, Other  ➜ fish11_clusters_overlay.pdf
import os, glob, ast, numpy as np, pandas as pd, matplotlib.pyplot as plt, tifffile

fish      = 11
COLOR_MAP = {"Pt": "orange", "Hb": "green", "Other": "purple"}

def bright_plane0(f, boost=1.5):
    img_dir = os.path.normpath(os.path.join(os.path.dirname(BASE_DIR), os.pardir,
                                            f"fish{f}_images"))
    plane0  = glob.glob(os.path.join(img_dir, "plane_0.*"))[0]
    im      = tifffile.imread(plane0).astype(float)
    im      = (im - im.min())/(im.max() - im.min())
    return np.clip(im*boost, 0, 1)

def plane0_table(f):
    h5 = os.path.normpath(os.path.join(os.path.dirname(BASE_DIR), os.pardir,
                                       f"fish{f}_images","functional_types_df.h5"))
    df = pd.read_hdf(h5)
    return df[df.plane == "plane_0"]

# load & preprocess
df0 = plane0_table(fish).copy()
df0["coords"] = df0.neur_coords.apply(
    lambda v: ast.literal_eval(v) if isinstance(v, str) else v
)
df0["grp"] = df0.region.fillna("unknown").apply(
    lambda r: r if r in ("Pt", "Hb") else "Other"
)

# plot
bg = bright_plane0(fish)
fig, ax = plt.subplots(figsize=(6,6), dpi=300)
ax.imshow(bg, cmap="gray", vmin=0, vmax=1)

for group, color in COLOR_MAP.items():
    coords_list = df0.loc[df0.grp == group, "coords"].tolist()
    if not coords_list:
        continue
    pts = np.vstack(coords_list)
    ax.scatter(pts[:,0], pts[:,1],
               s=40, c=color, edgecolors="white", alpha=.9,
               label=group)

ax.legend(loc="lower right")
ax.axis("off")
fig.patch.set_alpha(0); ax.patch.set_alpha(0)

out = os.path.join(BASE_DIR, f"fish{fish}_clusters_overlay.pdf")
fig.savefig(out, bbox_inches="tight", transparent=True)
plt.close(fig)
print("saved →", out)


saved → C:/Users/jacob/OneDrive/Documents/GitHub/external_data/nlp4neuro/results_april30/experiment_6\fish11_clusters_overlay.pdf


In [5]:
# Task 4 – bar‐plot Pt, Hb, Other + significance tests  
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.stats import sem, ttest_rel

fish_list = [11, 12, 13]
GROUPS    = ["Pt", "Hb", "Other"]
COLORS    = {"Pt": "orange", "Hb": "green", "Other": "purple"}

def load_saliency(f):
    # ❶ correct root – one experiment_6, not two
    root = os.path.join(BASE_DIR, f"fish{f}")      # ← FIXED
    print(root)
    vecs = [
        np.load(os.path.join(r, "importance.npy"))
        for r, _, fs in os.walk(root) if "importance.npy" in fs
    ]
    if not vecs:                                   # nice-to-have guard
        raise FileNotFoundError(
            f"No importance.npy files under {root}. "
            "Check the folder structure or BASE_DIR."
        )
    return np.vstack(vecs).mean(0)

def plane0_table(f):
    h5 = os.path.normpath(os.path.join(os.path.dirname(BASE_DIR), os.pardir,
                                       f"fish{f}_images","functional_types_df.h5"))
    df = pd.read_hdf(h5)
    return df[df.plane == "plane_0"]

# collect per‐fish group means
group_sals = []
for f in fish_list:
    sal = load_saliency(f)
    df  = plane0_table(f).copy()
    df["grp"] = df.region.fillna("unknown").apply(
        lambda r: r if r in ("Pt","Hb") else "Other"
    )
    vals = sal[df.index.astype(int)]
    s    = pd.Series(vals, index=df.grp).groupby(level=0).mean()
    group_sals.append(s)

# align to all three groups
df_groups = pd.concat(group_sals, axis=1).reindex(GROUPS)
means     = df_groups.mean(axis=1)
errs      = df_groups.apply(sem, axis=1)

# plot bars
x = np.arange(len(GROUPS))
fig, ax = plt.subplots(figsize=(5,2), dpi=300)
ax.bar(x, means, yerr=errs, capsize=5,
       color=[COLORS[g] for g in GROUPS])
ax.set_xticks(x)
ax.set_ylim(0.00045, 0.00095)
ax.set_xticklabels(GROUPS)
ax.set_ylabel("Mean saliency ± SEM")

# significance bracket helper
def draw_bracket(ax, x1, x2, y, h, text):
    ax.plot([x1, x1, x2, x2], [y, y+h, y+h, y],
            lw=1.5, c="black")
    ax.text((x1+x2)/2, y+h, text,
            ha="center", va="bottom", fontsize=12)

# run paired t-tests and annotate
print("Paired t-tests:")
offset = errs.max() * 0.1     # initial vertical offset
h      = errs.max() * 0.1     # bracket height
pairs = [(0,1), (0,2), (1,2)]
for i, j in pairs:
    grp_i, grp_j = GROUPS[i], GROUPS[j]
    vals_i, vals_j = df_groups.loc[grp_i], df_groups.loc[grp_j]
    t_stat, p_val = ttest_rel(vals_i, vals_j, nan_policy="omit")
    print(f"{grp_i} vs {grp_j}: p = {p_val:.4f}")
    if p_val < 0.2:
        star = "***" if p_val < 0.001 else "**" if p_val < 0.01 else f"p={round(p_val,1)}"
        y = max(means[i] + errs[i], means[j] + errs[j]) + offset
        draw_bracket(ax, i, j, y, h, star)
        offset += h * 2.8

fig.tight_layout()
out = os.path.join(BASE_DIR, "cluster_importance_barplot.pdf")
fig.savefig(out, bbox_inches="tight", transparent=True)
plt.close(fig)
print("saved →", out)



C:/Users/jacob/OneDrive/Documents/GitHub/external_data/nlp4neuro/results_april30/experiment_6\fish11
C:/Users/jacob/OneDrive/Documents/GitHub/external_data/nlp4neuro/results_april30/experiment_6\fish12
C:/Users/jacob/OneDrive/Documents/GitHub/external_data/nlp4neuro/results_april30/experiment_6\fish13
Paired t-tests:
Pt vs Hb: p = 0.1094
Pt vs Other: p = 0.4890
Hb vs Other: p = 0.4816
saved → C:/Users/jacob/OneDrive/Documents/GitHub/external_data/nlp4neuro/results_april30/experiment_6\cluster_importance_barplot.pdf


C:\Users\jacob\AppData\Local\Temp\ipykernel_48876\439071281.py:76: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = max(means[i] + errs[i], means[j] + errs[j]) + offset
